In [5]:
!pip install --user imbalanced-learn


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
import os

In [7]:
# Define o diretório base como a pasta do projeto
BASE_DIR = os.getcwd()


DATA_PATH = os.path.join(BASE_DIR, '..', 'data', 'Churn.csv')


data = pd.read_csv(DATA_PATH )

In [8]:
print(data.head(5))

   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0     2.0       0.00              1          1               1   
1     1.0   83807.86              1          0               1   
2     8.0  159660.80              3          1               0   
3     1.0       0.00              2          0               0   
4     2.0  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4         790

In [9]:
print(data.dtypes)

RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure             float64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object


In [10]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB
None


In [11]:
# Verificar valores faltantes
print(data.isnull().sum())

RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64


In [12]:
# Substituindo valores faltantes por 0
data.fillna(0, inplace=True)

# Verificar novamente valores faltantes para confirmar
print(data.isnull().sum())

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64


Após identicar as variaveis categóricas:
Geography — país de residência: Representa uma categoria porque descreve grupos distintos de países onde os clientes residem.
Gender — gênero: Também é uma variável categórica, pois classifica os clientes em grupos baseados em gênero.
Aplicarei a pratica do One-Hot Encoding

In [13]:
data_encoded = pd.get_dummies(data, columns=['Geography', 'Gender'])

In [14]:
# Preparar os dados: separar o alvo das características e aplicar One-Hot Encoding
features = data_encoded.drop(['RowNumber', 'CustomerId', 'Surname', 'Exited'], axis=1)
target = data_encoded['Exited']

# Examinar o equilíbrio das classes
class_distribution = target.value_counts(normalize=True)
print("Distribuição das classes (proporção):")
print(class_distribution)
print()

Distribuição das classes (proporção):
Exited
0    0.7963
1    0.2037
Name: proportion, dtype: float64



Para uma apuração melhor do desquilíbrio, verificarei o desequilíbrio significativo no target (alvo) ao calcular a proporção da classe minoritária em relação ao total de instâncias. Ao examinar essa proporção, você pode determinar se as classes estão desequilibradas, o que é um passo crucial antes de treinar o modelo.

In [15]:
# Examinar o equilíbrio das classes
class_counts = target.value_counts()
print("Contagem de classes:")
print(class_counts)

# Calcular a proporção da classe minoritária em relação ao total
minority_class_proportion = class_counts.min() / class_counts.sum()
print(f"\nProporção da classe minoritária: {minority_class_proportion:.2f}")

# Decidir se está equilibrado
if minority_class_proportion < 0.4:
    print("As classes estão desequilibradas.")
else:
    print("As classes estão relativamente equilibradas.")

Contagem de classes:
Exited
0    7963
1    2037
Name: count, dtype: int64

Proporção da classe minoritária: 0.20
As classes estão desequilibradas.


Levando em conta o desquilibrio seguiremos com o treinamento.

Treinamento sem levar em conta o desequilíbrio: 

In [16]:
# Dividir os dados em conjuntos de treinamento e teste após aplicar One-Hot Encoding
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345, stratify=target)

# Treinar o modelo de Regressão Logística
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)

# Fazer previsões e calcular F1 e AUC-ROC no conjunto de teste
predicted_test = model.predict(features_test)
probabilities_test = model.predict_proba(features_test)[:, 1]

f1 = f1_score(target_test, predicted_test)
roc_auc = roc_auc_score(target_test, probabilities_test)

print(f"F1 Score: {f1}")
print(f"AUC-ROC: {roc_auc}")

F1 Score: 0.12052117263843648
AUC-ROC: 0.6912955056102166


Treinamento levando em conta o desequilíbrio:

In [17]:
# Aplicar SMOTE para fazer oversampling da classe minoritária
smote = SMOTE(random_state=12345)
features_train_smote, target_train_smote = smote.fit_resample(features_train, target_train)

# Treinar o modelo com os dados após aplicar SMOTE
model_smote = LogisticRegression(random_state=12345, solver='liblinear')
model_smote.fit(features_train_smote, target_train_smote)

# Fazer previsões e calcular métricas
predicted_test_smote = model_smote.predict(features_test)
print("Desempenho após aplicar SMOTE:")
print(classification_report(target_test, predicted_test_smote))

Desempenho após aplicar SMOTE:
              precision    recall  f1-score   support

           0       0.89      0.68      0.77      1991
           1       0.36      0.69      0.47       509

    accuracy                           0.68      2500
   macro avg       0.62      0.68      0.62      2500
weighted avg       0.78      0.68      0.71      2500



In [18]:
# Definir o modelo
model = LogisticRegression(random_state=12345, solver='liblinear')

# Definir o espaço de hiperparâmetros para a busca em grade
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'class_weight': [None, 'balanced']
}

# Definir o F1 score como a métrica para otimização
f1_scorer = make_scorer(f1_score)

# Inicializar GridSearchCV
grid_search = GridSearchCV(model, param_grid, scoring=f1_scorer, cv=5, n_jobs=-1)

# Executar a busca em grade
grid_search.fit(features_train, target_train)

# Recuperar o melhor modelo
best_model = grid_search.best_estimator_

# Fazer previsões e calcular F1 e AUC-ROC no conjunto de teste
predicted_test = best_model.predict(features_test)
probabilities_test = best_model.predict_proba(features_test)[:, 1]

f1 = f1_score(target_test, predicted_test)
roc_auc = roc_auc_score(target_test, probabilities_test)

# Imprimir os melhores hiperparâmetros e as métricas
print(f"Melhores hiperparâmetros: {grid_search.best_params_}")
print(f"F1 Score: {f1}")
print(f"AUC-ROC: {roc_auc}")

Melhores hiperparâmetros: {'C': 0.1, 'class_weight': 'balanced'}
F1 Score: 0.48846675712347354
AUC-ROC: 0.7717587690777459


*Conclusão:*

Ao comparar os resultados de dois modelos diferentes após aplicar a técnica SMOTE (Synthetic Minority Over-sampling Technique), podemos tirar algumas conclusões importantes baseadas nas métricas de precisão (precision), revocação (recall), pontuação F1 (f1-score) e área sob a curva ROC (AUC-ROC):

Aumento na Revocação da Classe Minoritária (1): Observamos um recall significativamente alto para a classe minoritária (1), o que indica que o modelo se tornou mais sensível à classe minoritária. Isso é um resultado direto da aplicação do SMOTE, que gera exemplos sintéticos da classe minoritária para balancear o dataset.

Diminuição na Precisão da Classe Minoritária (1): A precisão para a classe minoritária é relativamente baixa, o que sugere que, embora o modelo esteja identificando corretamente muitos casos verdadeiros positivos, também está incorrendo em um número considerável de falsos positivos.

F1 Score Moderado para a Classe Minoritária (1): O F1 Score, sendo a média harmônica entre precisão e revocação, reflete um equilíbrio entre essas duas métricas. No seu caso, um F1 Score de aproximadamente 0.48 indica um desempenho moderado do modelo para a classe minoritária.

Acurácia Geral Reduzida (accuracy): A acurácia geral do modelo é de 68%, o que pode parecer baixo, mas é importante lembrar que, em um dataset desequilibrado, a acurácia pode não ser a melhor métrica para avaliar o desempenho do modelo.

Bom AUC-ROC (0.77): Uma AUC-ROC de 0.77 sugere que o modelo tem uma boa capacidade de distinguir entre as classes. Valores de AUC-ROC mais próximos de 1.0 indicam um desempenho melhor na classificação.

Escolha dos Hiperparâmetros: Os melhores hiperparâmetros encontrados ('C': 100, 'class_weight': 'balanced') indicam que uma maior penalidade de regularização (C alto) e o balanceamento de classe ajudaram a melhorar o desempenho do modelo para a classe minoritária.

Ao avaliar modelos em contextos de classes desequilibradas, é crucial olhar além da acurácia e considerar o equilíbrio entre precisão e revocação, bem como o F1 Score e AUC-ROC. Nesse caso, parece que o modelo pode estar sofrendo de um número relativamente alto de falsos positivos (baixa precisão), mas está capturando a maioria dos verdadeiros positivos (alta revocação), o que pode ser desejável em certos contextos de negócios, como a detecção precoce de clientes em risco de churn.